In [1]:
import warnings                        
warnings.filterwarnings('ignore')  
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
INFO:tensorflow:Disabling eager execution
INFO:tensorflow:Disabling v2 tensorshape
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Disabling resource variables
INFO:tensorflow:Disabling tensor equality
INFO:tensorflow:Disabling control flow v2


#### 학습데이터를 만든다.

In [2]:
# xData -> [공부 시간, 과외 시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10,7], [12, 8], [14, 9]]
print(temp)
print(type(temp))
xData = np.array(temp)
print(xData)

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]


In [3]:
# yData -> [합격 여부] -> 실제값
temp = [0, 0, 0, 1, 1, 1, 1]
yData = np.array(temp)
print(yData)
# reshpape : 넘파이에서 데이터는 그대로 유지한 채 배열의 형태(차원)을 변경한다. 1행 7열 넘파이 배열 -> 7행 1열 넘파이 배열
yData = np.array(temp).reshape(7,1)
print(yData)
print(type(yData))

[0 0 0 1 1 1 1]
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


#### xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해 대입되는 넘파이의 차원을 지정해줘야 한다.
# [None, 2] -> placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관이 없고 열의 개수는 무조건 2개이다.
                        # [None, None] 행, 열의 개수는 몇개라도 상관없다.
                        # [10,2] -> 10행 2열
X = tf.placeholder(dtype=tf.float32, shape = [None,2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape = [None,1]) # ydata가 대입될 placeholder

#### 기울기(가중치)와 y절편(바이어스) b를 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2,1],dtype=tf.float32)) # 난수를 2행1열로 발생시킨다. 기울기
b = tf.Variable(tf.random_uniform([1],dtype=tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {}, a2:{}, b: {}'.format(sess.run(a)[0],sess.run(a)[1],sess.run(b)))
print('='*60, '\n','a')
print('{}'.format(sess.run(a)))
print('='*60,'\n','b')
print('{}'.format(sess.run(b)))

a1: [0.67597616], a2:[0.60612214], b: [0.69739544]
 a
[[0.67597616]
 [0.60612214]]
 b
[0.69739544]


#### 시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# matmul() : tensorflow에서 행렬의 곱셈을 계산하는 메소드
# sigmoid() : tensorflow에서 시그모이드 방정식을 계산하는 메소드

y = tf.sigmoid(tf.matmul(X, a) + b)
loss = -tf.reduce_mean(Y*tf.log(y) + (1 - Y) * tf.log(1-y)) # 오차함수
gradient_descent = tf.train.GradientDescentOptimizer(0.01).minimize(loss) # 경사 하강

#### sigmoid() 메소드의 실행 결과(y)가 0.5이상이면 1을 0.5미만이면 0을 리턴한다.

In [7]:
sess = tf.Session()
# cast(캐스팅할 데이터, dtype = 캐스팅 될 데이터 타입) : tensorflow에서 형변환을 실행하는 베소드
predicted = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32) # 실수를 정수 데이터로 변환한다.
#print(sess.run(predicted)) # [1,2]
# cast() 메소드의 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅한다.
predicted = tf.cast(0.4>=0.5,dtype=tf.int32)
print(sess.run(predicted)) # 0
predicted = tf.cast(0.5>=0.5,dtype=tf.int32) 
print(sess.run(predicted)) # 1

0
1


#### sigmoid() 메소드를 실행한 예상값을 계산한다.

In [8]:
predicted = tf.cast(y >= 0.5, dtype=tf.float32)  # 예측값, sigmoid() 메소드를 실행한 결과(y)를 0또는 1로 변환한다.
# 예측값와 실제값이 일치하는 정도(정확도)를 계산한다.
# equal() : tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 메소드
# equal() 메소드로 예측값(predicted)과 실제값(Y)이 같은지 비교 후 True 또는 False를 1 또는 0으로 캐스팅하고 시행된 전체 평균을 
# 계산한다.
accuracy  = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.int32))

#### 학습시킨다.

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001):
    # sess.run([시울기, y절편, 오차함수, 경사하강],feed_dict = {X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    a_, b_,loss_, _ = sess.run([a,b,loss,gradient_descent], feed_dict={X:xData, Y: yData})
    if i % 300 == 0:
        # Epoch: 학습 횟수, a1: 공부 시간 기울기, a2: 과외 시간 기울기, b: y절편, loss: 오차 함수
        print('Epoch: %4d, a1: %9.5f, a2: %9.5f, b: %9.5f, loss: %9.5f' %(i, a_[0], a_[1], b_, loss_))
        
# 데스트 데이터를 만든다.
#new_x = np.array([6,5]) # 테스트 데이터가 1차원 넘파이 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
#print(new_x) # [6,5] -> 1차원
# reshape() 메소드를 이용해서 1행 2열인 2차원 넘파이 배열로 테스트 데이터를 만들어야 한다.
new_x = np.array([6,5]).reshape(1,2)
print(new_x) # [[6 5]] -> 2차원

# sigmoid() 메소드의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predicted,y],feed_dict={X: new_x})
print('공부시간: %d, 과외시간: %d' %(new_x[:, 0],new_x[:,1]))
print('합격 여부: %d, 합격 확률: %f' %(result, new_y))
print('='*80)

for i in range(11):
    new_x = np.array([0,i]).reshape(1,2)
    result, new_y = sess.run([predicted,y],feed_dict={X: new_x})
    print('공부시간: %d, 과외시간: %2d' %(new_x[:, 0],new_x[:,1]), end = ' ')
    print('합격 여부: %s, 합격 확률: %f%%' %(' 합격' if result==1 else '불합격', new_y*100))

Epoch:    0, a1:   0.64149, a2:   0.40485, b:   0.57330, loss:   1.98261
Epoch:  300, a1:   0.30411, a2:  -0.25392, b:   0.02529, loss:   0.52404
Epoch:  600, a1:   0.46299, a2:  -0.41582, b:  -0.36856, loss:   0.45472
Epoch:  900, a1:   0.57445, a2:  -0.51749, b:  -0.70519, loss:   0.40915
Epoch: 1200, a1:   0.65462, a2:  -0.58036, b:  -1.00166, loss:   0.37630
Epoch: 1500, a1:   0.71360, a2:  -0.61749, b:  -1.26865, loss:   0.35086
Epoch: 1800, a1:   0.75761, a2:  -0.63684, b:  -1.51300, loss:   0.33016
Epoch: 2100, a1:   0.79067, a2:  -0.64347, b:  -1.73923, loss:   0.31270
Epoch: 2400, a1:   0.81545, a2:  -0.64073, b:  -1.95054, loss:   0.29759
Epoch: 2700, a1:   0.83383, a2:  -0.63094, b:  -2.14923, loss:   0.28428
Epoch: 3000, a1:   0.84714, a2:  -0.61574, b:  -2.33706, loss:   0.27238
[[6 5]]
공부시간: 6, 과외시간: 5
합격 여부: 0, 합격 확률: 0.417530
공부시간: 0, 과외시간:  0 합격 여부: 불합격, 합격 확률: 8.809957%
공부시간: 0, 과외시간:  1 합격 여부: 불합격, 합격 확률: 4.960394%
공부시간: 0, 과외시간:  2 합격 여부: 불합격, 합격 확률: 2.742335%
공부시간: